In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df3 = pd.read_table('./data/TWR_C_2nd_cook_03m_fft1.txt', delim_whitespace=True, index_col=0,
                  names=['Frac', 'U', 'V', 'W', 'T', 'year', 'mon', 'day', 'hour', 'min', 'tenth'])
df10 = pd.read_table('./data/TWR_C_2nd_cook_10m_fft1.txt', delim_whitespace=True, index_col=0,
                  names=['Frac', 'U', 'V', 'W', 'T', 'year', 'mon', 'day', 'hour', 'min', 'tenth'])
df20 = pd.read_table('./data/TWR_C_2nd_cook_20m_fft1.txt', delim_whitespace=True, index_col=0,
                  names=['Frac', 'U', 'V', 'W', 'T', 'year', 'mon', 'day', 'hour', 'min', 'tenth'])

In [11]:
# start = ['15:08:00', '14:55:00', '15:43:00', '14:25:00', '14:55:00']
# ffps = ['15:38:00', '15:25:00', '16:13:00', '14:55:00', '15:25:00']
# ffpe = ['15:53:00', '15:40:00', '16:33:00', '15:15:00', '15:45:00']
# end = ['16:23:00', '16:10:00', '17:03:00', '15:45:00', '16:15:00']
tower = ['East', 'Flux', 'North', 'South_Mobile', 'West']

cstart = [543231, 535431, 564231, 517431, 535431]
cffps = [561231, 553431, 582231, 535431, 553431]
cffpe = [570231, 562431, 594231, 547431, 565431]
cend = [588231, 580431, 612231, 565431, 583431]
# df = df3
# df[np.logical_and(df['hour']==15, df['min']==45)]
# df.iloc[565431:]
ti = 0
# df.iloc[cstart[ti]:cffps[ti]]
# df.iloc[cffps[ti]:cffpe[ti]]
# df.iloc[cffpe[ti]:cend[ti]]

In [4]:
# # period mean 
# t = []
# w = []
# wt = []
# tmean = []
# wmean = []
# wtmean = []

# df = [df3, df10, df20]
# for hi in range(3):
#     df_pre = df[hi].iloc[cstart[ti]:cffps[ti]] #pre
#     df_ffp = df[hi].iloc[cffps[ti]:cffpe[ti]] #ffp
#     df_pos = df[hi].iloc[cffpe[ti]:cend[ti]] #post
    
#     t.append(df_pre['T'] - df_pre['T'].mean())
#     t.append(df_ffp['T'] - df_ffp['T'].mean())
#     t.append(df_pos['T'] - df_pos['T'].mean())
    
#     w.append(df_pre['W'] - df_pre['W'].mean())
#     w.append(df_ffp['W'] - df_ffp['W'].mean())
#     w.append(df_pos['W'] - df_pos['W'].mean())

#     wt.append((df_pre['T'] - df_pre['T'].mean()) * (df_pre['W'] - df_pre['W'].mean()))
#     wt.append((df_ffp['T'] - df_ffp['T'].mean()) * (df_ffp['W'] - df_ffp['W'].mean()))
#     wt.append((df_pos['T'] - df_pos['T'].mean()) * (df_pos['W'] - df_pos['W'].mean()))
    
#     tmean.append([df_pre['T'].mean(), df_ffp['T'].mean(), df_pos['T'].mean()])
#     wmean.append([df_pre['W'].mean(), df_ffp['W'].mean(), df_pos['W'].mean()])
#     wtmean.append([((df_pre['T'] - df_pre['T'].mean()) * (df_pre['W'] - df_pre['W'].mean())).mean(),
#                   ((df_ffp['T'] - df_ffp['T'].mean()) * (df_ffp['W'] - df_ffp['W'].mean())).mean(),
#                   ((df_pos['T'] - df_pos['T'].mean()) * (df_pos['W'] - df_pos['W'].mean())).mean()])

# data = [t, w, wt]

In [4]:
# period mean for pre and post and pre mean for ffp
t = []
w = []
wt = []
tmean = []
wmean = []
wtmean = []

df = [df3, df10, df20]
for hi in range(3):
    df_pre = df[hi].iloc[cstart[ti]:cffps[ti]] #pre
    df_ffp = df[hi].iloc[cffps[ti]:cffpe[ti]] #ffp
    df_pos = df[hi].iloc[cffpe[ti]:cend[ti]] #post
    
    t.append(df_pre['T'] - df_pre['T'].mean())
    t.append(df_ffp['T'] - df_pre['T'].mean())
    t.append(df_pos['T'] - df_pos['T'].mean())
    
    w.append(df_pre['W'] - df_pre['W'].mean())
    w.append(df_ffp['W'] - df_pre['W'].mean())
    w.append(df_pos['W'] - df_pos['W'].mean())

    wt.append((df_pre['T'] - df_pre['T'].mean()) * (df_pre['W'] - df_pre['W'].mean()))
    wt.append((df_ffp['T'] - df_pre['T'].mean()) * (df_ffp['W'] - df_pre['W'].mean()))
    wt.append((df_pos['T'] - df_pos['T'].mean()) * (df_pos['W'] - df_pos['W'].mean()))
    
    tmean.append([df_pre['T'].mean(), df_ffp['T'].mean(), df_pos['T'].mean()])
    wmean.append([df_pre['W'].mean(), df_ffp['W'].mean(), df_pos['W'].mean()])
    wtmean.append([((df_pre['T'] - df_pre['T'].mean()) * (df_pre['W'] - df_pre['W'].mean())).mean(),
                  ((df_ffp['T'] - df_ffp['T'].mean()) * (df_ffp['W'] - df_ffp['W'].mean())).mean(),
                  ((df_pos['T'] - df_pos['T'].mean()) * (df_pos['W'] - df_pos['W'].mean())).mean()])

data = [t, w, wt]

In [61]:
# tmean

In [7]:
datamean = [tmean, wmean, wtmean]
tower = ['East', 'Flux', 'North', 'South', 'West']
color = ['r', 'b', 'lime']
start = ['15:08:00', '14:55:00', '15:43:00', '14:25:00', '14:55:00']
ffps = ['15:38:00', '15:25:00', '16:13:00', '14:55:00', '15:25:00']
ffpe = ['15:53:00', '15:40:00', '16:33:00', '15:15:00', '15:45:00']
end = ['16:23:00', '16:10:00', '17:03:00', '15:45:00', '16:15:00']

ylabel = ["t' (\u2103)", "$w' (ms^{-1})$", "$w't' (\u2103ms^{-1})$"]
meantext = ["$\overline{t}$ = ", "$\overline{w}$ = ", "$\overline{w^{'}t^{'}}$ = "]
yl = -5, -8, -2
yr = 5, 8, 4
step = 2, 4, 2

legend = ['19m AGL', '10m AGL', 'Fire Starts', 'Fire Ends']
fig, axs = plt.subplots(3, 1, figsize=(15, 10))
for vi in range(3):
    ax = plt.subplot(3,1,vi+1)
    ax.set_ylim(yl[vi], yr[vi])
    plt.yticks(np.arange(yl[vi], yr[vi]+1, step[vi]), fontsize='large')
    ax.set_xlim(-10, 485)
    ax.set_ylabel(ylabel[vi], fontsize=15)
    plt.subplots_adjust(hspace=0.08)
    if vi == 2:
        plt.text(.5, -.16, 'Time', fontsize=15, fontweight='bold', c='k', ha='center', va='center', transform=ax.transAxes)
    for hi in [2,1]:
        wwpre = data[vi][3*hi]
        xpre = len(wwpre)

        wwffp = data[vi][3*hi+1]
        xffp = len(wwffp)

        wwpos = data[vi][3*hi+2]
        xpos = len(wwpos)

        plt.scatter(np.arange(xpre), wwpre, .5, color[hi], alpha=1.)
        plt.scatter(np.arange(xpre, xpre+xffp), wwffp, .5, color[hi], alpha=1., label='_nolegend_')
        plt.scatter(np.arange(xpre+xffp, xpre+xffp+xpos), wwpos, .5, color[hi], alpha=1., label='_nolegend_')
        
    plt.text(.03, .1, meantext[vi], fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
#     plt.text(.08, .1, f'{datamean[vi][0][0]:.3f}', fontsize=15, c=color[0], ha='center', va='center', transform=ax.transAxes)
    plt.text(.15, .1, f'{datamean[vi][1][0]:.3f}', fontsize=15, c=color[1], ha='center', va='center', transform=ax.transAxes)
    plt.text(.22, .1, f'{datamean[vi][2][0]:.3f}', fontsize=15, c=color[2], ha='center', va='center', transform=ax.transAxes)
#     plt.text(.44, .1, f'{datamean[vi][0][1]:.3f}', fontsize=15, c=color[0], ha='center', va='center', transform=ax.transAxes)
    plt.text(.51, .1, f'{datamean[vi][1][1]:.3f}', fontsize=15, c=color[1], ha='center', va='center', transform=ax.transAxes)
    plt.text(.58, .1, f'{datamean[vi][2][1]:.3f}', fontsize=15, c=color[2], ha='center', va='center', transform=ax.transAxes)
#     plt.text(.75, .1, f'{datamean[vi][0][2]:.3f}', fontsize=15, c=color[0], ha='center', va='center', transform=ax.transAxes)
    plt.text(.82, .1, f'{datamean[vi][1][2]:.3f}', fontsize=15, c=color[1], ha='center', va='center', transform=ax.transAxes)
    plt.text(.9, .1, f'{datamean[vi][2][2]:.3f}', fontsize=15, c=color[2], ha='center', va='center', transform=ax.transAxes)
    
    plt.vlines(xpre, yl[vi], yr[vi], 'magenta', linestyle='--', lw=1)
    plt.vlines(xpre+xffp, yl[vi], yr[vi], 'cyan', linestyle='--', lw=1) 
    plt.hlines(0, 0, xpre+xffp+xpos, 'grey', lw=1)
    plt.xticks(np.arange(0, xpre+xffp+xpos, 1000), [])
    if vi == 2:
        plt.text(0, -.08, start[ti], fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
#         plt.text(.21, -.08, mid1, fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
        plt.text(.41, -.08, ffps[ti], fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
        plt.text(.62, -.08, ffpe[ti], fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
#         plt.text(.82, -.08, mid2, fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
        plt.text(1, -.08, end[ti], fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
    if vi == 0:
        plt.legend(legend, fontsize='large', markerscale=3., ncol=2, frameon=False, loc='upper right')
    
plt.suptitle(f"Control Tower", fontsize=15, fontweight='bold', x=.5, y=.9)
plt.savefig(f'./plot/MixedMean/{tower[ti]}/Control_TimeSeries_mixedmean.png', bbox_inches='tight')
# plt.savefig(f'./plot/PeriodMean/{tower[ti]}/Control_TimeSeries_periodmean.png', bbox_inches='tight')
plt.close()

## scatter plot

In [49]:
# only run for scatter plot
t = []
w = []

df = [df20, df10, df3]
for hi in range(3):
    df_pre = df[hi].iloc[cstart[ti]:cffps[ti]] #pre
    df_ffp = df[hi].iloc[cffps[ti]:cffpe[ti]] #ffp
    df_pos = df[hi].iloc[cffpe[ti]:cend[ti]] #post
    
    t.append(df_pre['T'] - df_pre['T'].mean())
    t.append(df_ffp['T'] - df_pre['T'].mean())
    t.append(df_pos['T'] - df_pos['T'].mean())
    
    w.append(df_pre['W'] - df_pre['W'].mean())
    w.append(df_ffp['W'] - df_pre['W'].mean())
    w.append(df_pos['W'] - df_pos['W'].mean())

In [37]:
SweepC = [np.logical_and(w[i]<0, t[i]<0).sum() for i in range(9)]
EjectionC = [np.logical_and(w[i]>0, t[i]>0).sum() for i in range(9)]
OutwardC = [np.logical_and(w[i]>0, t[i]<0).sum() for i in range(9)]
InwardC = [np.logical_and(w[i]<0, t[i]>0).sum() for i in range(9)]

TotalC = [SweepC[i]+EjectionC[i]+OutwardC[i]+InwardC[i] for i in range(9)]
SweepMean = [np.where(np.logical_and(w[i]<0, t[i]<0), w[i]*t[i], 0).sum() / TotalC[i] for i in range(9)]
EjectionMean = [np.where(np.logical_and(w[i]>0, t[i]>0), w[i]*t[i], 0).sum() / TotalC[i] for i in range(9)]
OutwardMean = [np.where(np.logical_and(w[i]>0, t[i]<0), w[i]*t[i], 0).sum() / TotalC[i] for i in range(9)]
InwardMean = [np.where(np.logical_and(w[i]<0, t[i]>0), w[i]*t[i], 0).sum() / TotalC[i] for i in range(9)]

OutwardFra = [OutwardC[i] / TotalC[i] for i in range(9)]
EjectionFra = [EjectionC[i] / TotalC[i] for i in range(9)]
InwardFra = [InwardC[i] / TotalC[i] for i in range(9)]
SweepFra = [SweepC[i] / TotalC[i] for i in range(9)]

In [38]:
print(f'{tower[ti]}')
[OutwardFra[i] + InwardFra[i] + EjectionFra[i] + SweepFra[i] for i in range(9)]

West


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [52]:
color = ['lime', 'r', 'k']
xl = [-4, -4, -4]
xr = [4, 4, 4]
yl = [-4, -4, -4]
yr = [4, 4, 4]
fig, axs = plt.subplots(3, 3, figsize=(12, 12))
for hi in range(3):
    for pi in range(3):
        ax = plt.subplot(3,3,hi*3+pi+1)
        plt.scatter(t[hi*3+pi], w[hi*3+pi], c=color[pi], s=1)
        plt.xlim(xl[pi], xr[pi])
        plt.ylim(yl[pi], yr[pi])
        plt.hlines(0, xl[pi], xr[pi], 'c', lw=1)
        plt.vlines(0, yl[pi], yr[pi], 'c', lw=1)
        plt.subplots_adjust(hspace=.06, wspace=.06)
        if pi == 0:
            plt.ylabel("$w' (ms^{-1})$", fontsize=15)
            plt.yticks([-4, -2, 0, 2, 4], fontsize=13)
        else:
            plt.yticks([-4, -2, 0, 2, 4], [])
        if hi == 2:
            plt.xlabel("t' (\u2103)", fontsize=15)
            plt.xticks([-4, -2, 0, 2, 4], fontsize=13)
        else:
            plt.xticks([-4, -2, 0, 2, 4], [])
        if hi == 0:
            if pi == 0:
                plt.text(.5, 1.08, 'Pre-FFP', fontsize='large', fontweight='bold', c='lime', ha='center', va='center', transform=ax.transAxes)
            elif pi == 1:
                plt.text(.5, 1.08, 'FFP', fontsize='large', fontweight='bold', c='r', ha='center', va='center', transform=ax.transAxes)
            elif pi == 2:
                plt.text(.5, 1.08, 'Post-FFP', fontsize='large', fontweight='bold', c='k', ha='center', va='center', transform=ax.transAxes)
        
        if pi == 0:
            if hi == 0:
                plt.text(-.17, .9, '19-m', fontsize=15, ha='center', va='center', transform=ax.transAxes)
            elif hi == 1:
                plt.text(-.17, .9, '10-m', fontsize=15, ha='center', va='center', transform=ax.transAxes)
            elif hi == 2:
                plt.text(-.17, .9, '3-m', fontsize=15, ha='center', va='center', transform=ax.transAxes)
        
        plt.text(.25, .96, 'Outward', c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.23, .89, "$\overline{w^{'}t^{'}}$ = "+f"{OutwardMean[hi*3+pi]:.4f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.23, .81, f" Fraction= {OutwardFra[hi*3+pi]:.2f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.25, .2, 'Sweep', c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.23, .12, "$\overline{w^{'}t^{'}}$ = "+f"{SweepMean[hi*3+pi]:.4f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.23, .05, f" Fraction= {SweepFra[hi*3+pi]:.2f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.77, .2, 'Inward', c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.77, .12, "$\overline{w^{'}t^{'}}$ = "+f"{InwardMean[hi*3+pi]:.4f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.75, .05, f" Fraction= {InwardFra[hi*3+pi]:.2f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.77, .96, 'Ejection', c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.77, .89, "$\overline{w^{'}t^{'}}$ = "+f"{EjectionMean[hi*3+pi]:.4f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.75, .81, f" Fraction= {EjectionFra[hi*3+pi]:.2f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
plt.suptitle('Control tower', fontsize=15, fontweight='bold', x=.5, y=.93)
plt.savefig(f'./plot/MixedMean/{tower[ti]}/Control_scatterplot_mixedmean.png', bbox_inches='tight')
# plt.savefig(f'./plot/PeriodMean/{tower[ti]}/Control_scatterplot_periodmean.png', bbox_inches='tight')
plt.close()

In [38]:
SC = np.array(SweepFra)
EC = np.array(EjectionFra)
OC = np.array(OutwardFra)
IC = np.array(InwardFra)
HeatMean = np.array(SweepMean) + np.array(EjectionMean) + np.array(InwardMean) + np.array(OutwardMean)

contriF = [SC, EC, OC, IC, HeatMean]
PerChge = np.zeros((6, 5))
for ei in range(5):
    for hi in range(3):
        PerChge[hi*2, ei] = (contriF[ei][hi*3+1] - contriF[ei][hi*3]) / contriF[ei][hi*3] * 100
        PerChge[hi*2+1, ei] = (contriF[ei][hi*3+2] - contriF[ei][hi*3]) / contriF[ei][hi*3] * 100

In [39]:
print(f'{tower[ti]}')
pd.DataFrame({'Sweep':SC, 'Ejection':EC, 'Outward':OC, 'Inward':IC, 'Total':HeatMean}).T.round(2)

South_Mobile


,0,1,2,3,4,5,6,7,8
Sweep,0.23,0.07,0.25,0.32,0.09,0.29,0.42,0.18,0.39
Ejection,0.24,0.57,0.28,0.36,0.61,0.35,0.35,0.55,0.35
Outward,0.29,0.08,0.26,0.22,0.04,0.22,0.15,0.03,0.13
Inward,0.24,0.28,0.21,0.10,0.26,0.14,0.08,0.24,0.12
Total,0.01,0.12,0.02,0.15,0.23,0.09,0.26,0.48,0.19


In [40]:
print(f'{tower[ti]}')
table = pd.DataFrame({'Sweep':PerChge[:, 0], 'Ejection':PerChge[:, 1], 'Outward':PerChge[:, 2], 'Inward':PerChge[:, 3], 'Total':PerChge[:, 4]}).T.round(2).astype(str)
table.style.applymap(color_PositiveNegative)

South_Mobile


,0,1,2,3,4,5
Sweep,-67.51,8.24,-71.74,-9.97,-57.35,-6.95
Ejection,136.42,15.35,71.98,-2.67,56.9,1.62
Outward,-72.85,-10.58,-81.77,0.71,-79.19,-10.1
Inward,15.33,-10.66,155.4,40.09,202.4,48.5
Total,1814.95,274.3,58.31,-41.32,85.65,-26.98


### Average through control tower

In [10]:
# EventMean = np.zeros((3, 5, 3, 4))
# Frac = np.zeros((3, 5, 3, 4))

# # height(3,10,20), tower(East, Flux, North, South, West), period(Pre, FFP, Post), event(Sweep, Ejection, Inward, Outward)
# SC = np.array(SweepMean)
# EC = np.array(EjectionMean)
# OC = np.array(OutwardMean)
# IC = np.array(InwardMean)
# for hi in range(3):
#     EventMean[hi, ti, :, 0] = SC[hi*3:hi*3+3]
#     EventMean[hi, ti, :, 1] = EC[hi*3:hi*3+3]
#     EventMean[hi, ti, :, 2] = IC[hi*3:hi*3+3]
#     EventMean[hi, ti, :, 3] = OC[hi*3:hi*3+3]

# SC = np.array(SweepFra)
# EC = np.array(EjectionFra)
# OC = np.array(OutwardFra)
# IC = np.array(InwardFra)
# for hi in range(3):
#     Frac[hi, ti, :, 0] = SC[hi*3:hi*3+3]
#     Frac[hi, ti, :, 1] = EC[hi*3:hi*3+3]
#     Frac[hi, ti, :, 2] = IC[hi*3:hi*3+3]
#     Frac[hi, ti, :, 3] = OC[hi*3:hi*3+3]

# EventMean = np.nanmean(EventMean, axis=1)
# HeatMean = EventMean.sum(axis=2)
# Frac = np.nanmean(Frac, axis=1)

In [2]:
contri = np.load('./data/ContributionValue_Control.npy') #(5, 4, 9)
fracti = np.load('./data/FractionValue_Control.npy')

In [11]:
EventMean = np.zeros((3, 5, 3, 4))
Frac = np.zeros((3, 5, 3, 4))
for hi in range(3):
    for ti in range(5):
        for pi in range(3):
            for ei in range(4):
                if ei<2:
                    EventMean[hi, ti, pi, ei] = contri[ti, ei, hi*3+pi]
                    Frac[hi, ti, pi, ei] = fracti[ti, ei, hi*3+pi]
                elif ei == 2:
                    EventMean[hi, ti, pi, ei] = contri[ti, ei+1, hi*3+pi]
                    Frac[hi, ti, pi, ei] = fracti[ti, ei+1, hi*3+pi]
                elif ei == 3:
                    EventMean[hi, ti, pi, ei] = contri[ti, ei-1, hi*3+pi]
                    Frac[hi, ti, pi, ei] = fracti[ti, ei-1, hi*3+pi]
                
EventMean = np.nanmean(EventMean, axis=1)
HeatMean = EventMean.sum(axis=2)
Frac = np.nanmean(Frac, axis=1)

In [12]:
# contri[:, 0, 4]

In [16]:
# height(3,10,20), period(Pre, FFP, Post), event(Sweep, Ejection, Inward, Outward)
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
Y = [10, 19]
for pi in range(3):
    ax = plt.subplot(1,3,pi+1)
    plt.barh(Y, EventMean[1:, pi, 0]+EventMean[1:, pi, 1], align='center', height=2, color='pink')
    plt.barh(Y, EventMean[1:, pi, 0], align='center', height=2, color='red')
    plt.barh(Y, EventMean[1:, pi, 2]+EventMean[1:, pi, 3], align='center', height=2, color='b')
    plt.barh(Y, EventMean[1:, pi, 2], align='center', height=2, color='cyan')
#     plt.legend(['Ejection', 'Sweep', 'Inward Interaction', 'Outward Interaction'], loc='best', frameon=False)
#     plt.vlines(HeatMean[0, pi], 1.5, 4.5, lw=1, colors='yellow')
#     plt.text(HeatMean[0, pi]-.018, 4.7, f'{HeatMean[0, pi]:.3f}', fontsize=14)
    plt.vlines(HeatMean[1, pi], 8.5, 11.5, lw=1, colors='yellow')
    plt.text(HeatMean[1, pi]-.018, 11.7, f'{HeatMean[1, pi]:.3f}', fontsize=14)
    plt.vlines(HeatMean[2, pi], 17.5, 20.5, lw=1, colors='yellow')
    plt.text(HeatMean[2, pi]-.018, 20.7, f'{HeatMean[2, pi]:.3f}', fontsize=14)
    plt.vlines(0, 0, 22, lw=1, ls='dashed', color='grey')
    plt.xlim(-.4, .7)
    plt.xticks([i*.1 for i in range(-4, 7)], ['-0.4', '', '-0.2', '', '0.0', '', '0.2', '', '0.4', '', '0.6', ''], fontsize=14)
    plt.xlabel("$w't' (\u2103ms^{-1})$", fontsize=15)
    plt.ylim(0, 22)
    if pi == 0:
        plt.ylabel("Height (m)", fontsize=15)
        plt.yticks([i for i in range(0, 22, 3)], fontsize=14)
    else:
        plt.yticks([i for i in range(0, 22, 3)], [])
    
    if pi == 0:
        plt.text(.5, 1.2, 'Pre-FFP', fontsize=15, fontweight='bold', c='lime', ha='center', va='center', transform=ax.transAxes)
    elif pi == 1:
        plt.text(.5, 1.2, 'FFP', fontsize=15, fontweight='bold', c='r', ha='center', va='center', transform=ax.transAxes)
    elif pi == 2:
        plt.text(.5, 1.2, 'Post-FFP', fontsize=15, fontweight='bold', c='k', ha='center', va='center', transform=ax.transAxes)
    
    axx = ax.twiny()
    axx.plot(Frac[1:, pi, 0], Y, 'r-o', ms=3)
    axx.scatter(Frac[1:, pi, 0], Y, color='m')
    axx.plot(Frac[1:, pi, 1], Y, c='pink', ls='-', marker='o', ms=3)
    axx.scatter(Frac[1:, pi, 1], Y, color='m')
    axx.plot(Frac[1:, pi, 2], Y, c='cyan', ls='-', marker='o', ms=3)
    axx.scatter(Frac[1:, pi, 2], Y, color='m')
    axx.plot(Frac[1:, pi, 3], Y, 'b-o', ms=3)
    axx.scatter(Frac[1:, pi, 3], Y, color='m')
    axx.set_xlim(0, .7)
    axx.set_xticks(np.arange(0, 0.7, 0.1))
    axx.set_xticklabels(['0.0', '', '0.2', '', '0.4', '', '0.6', ''], fontsize=14)
    axx.set_xlabel('Fraction of Events', fontsize=15)
    axx.set_yticks(np.arange(0,22,3))
    plt.legend(['Sweep', 'Ejection', 'Inward', 'Outward'], loc='lower right', frameon=False, fontsize=14)
plt.savefig('./plot/ContributionFreq_AverControlTower.png', bbox_inches='tight')
plt.close()
# plt.show()

## Contribution bar and vertical profile

In [48]:
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
Y = [3, 10, 19]
HeatMean = np.array(SweepMean) + np.array(EjectionMean) + np.array(InwardMean) + np.array(OutwardMean)
for pi in range(3):
    ax = plt.subplot(1,3,pi+1)
    plt.subplots_adjust(wspace=.06)
    plt.barh(Y, np.array(SweepMean[pi::3])+np.array(EjectionMean[pi::3]), align='center', height=2, color='pink')
    plt.barh(Y, np.array(SweepMean[pi::3]), align='center', height=2, color='red')
    plt.barh(Y, np.array(InwardMean[pi::3])+np.array(OutwardMean[pi::3]), align='center', height=2, color='b')
    plt.barh(Y, np.array(InwardMean[pi::3]), align='center', height=2, color='cyan')
#     plt.legend(['Ejection', 'Sweep', 'Inward Interaction', 'Outward Interaction'], loc='best', frameon=False)
    plt.vlines(HeatMean[pi], 1.5, 4.5, lw=1, colors='yellow')
    plt.text(HeatMean[pi]-.018, 4.7, f'{HeatMean[pi]:.3f}', fontsize=14)
    plt.vlines(HeatMean[pi+3], 8.5, 11.5, lw=1, colors='yellow')
    plt.text(HeatMean[pi+3]-.018, 11.7, f'{HeatMean[pi+3]:.3f}', fontsize=14)
    plt.vlines(HeatMean[pi+6], 17.5, 20.5, lw=1, colors='yellow')
    plt.text(HeatMean[pi+6]-.018, 20.7, f'{HeatMean[pi+6]:.3f}', fontsize=14)
    plt.vlines(0, 0, 22, lw=1, ls='dashed', color='grey')
    plt.xlim(-.1, .7)
    plt.ylim(0, 22)
    plt.xticks([i*.1 for i in range(-1, 7)], ['', '0.0', '', '0.2', '', '0.4', '', '0.6', ''], fontsize=14)
    plt.xlabel("$w't' (\u2103ms^{-1})$", fontsize=15)
    if pi == 0:
        plt.ylabel("Height (m)", fontsize=15)
        plt.yticks([i for i in range(0, 22, 3)], fontsize=14)
    else:
        plt.yticks([i for i in range(0, 22, 3)], [])
    
    if pi == 0:
        plt.text(.5, 1.2, 'Pre-FFP', fontsize=15, fontweight='bold', c='lime', ha='center', va='center', transform=ax.transAxes)
    elif pi == 1:
        plt.text(.5, 1.2, 'FFP', fontsize=15, fontweight='bold', c='r', ha='center', va='center', transform=ax.transAxes)
    elif pi == 2:
        plt.text(.5, 1.2, 'Post-FFP', fontsize=15, fontweight='bold', c='k', ha='center', va='center', transform=ax.transAxes)
    
    axx = ax.twiny()
    axx.plot(SweepFra[pi::3], Y, 'r-o', ms=3)
    axx.scatter(SweepFra[pi::3], Y, color='m')
    axx.plot(EjectionFra[pi::3], Y, c='pink', ls='-', marker='o', ms=3)
    axx.scatter(EjectionFra[pi::3], Y, color='m')
    axx.plot(InwardFra[pi::3], Y, c='cyan', ls='-', marker='o', ms=3)
    axx.scatter(InwardFra[pi::3], Y, color='m')
    axx.plot(OutwardFra[pi::3], Y, 'b-o', ms=3)
    axx.scatter(OutwardFra[pi::3], Y, color='m')
    axx.set_xlim(0, .7)
    axx.set_xticks(np.arange(0, 0.7, 0.1))
    axx.set_xticklabels(['0.0', '', '0.2', '', '0.4', '', '0.6', ''], fontsize=14)
    axx.set_xlabel('Fraction of Events', fontsize=15)
    axx.set_yticks(np.arange(0,22,3))
    plt.legend(['Sweep', 'Ejection', 'Inward', 'Outward'], loc='lower right', frameon=False, fontsize=14)
plt.suptitle('Control tower', fontsize=15, fontweight='bold', x=.5, y=1.12)
plt.savefig(f'./plot/MixedMean/{tower[ti]}/Control_ContributionProfile_mixedmean.png', bbox_inches='tight')
# plt.savefig(f'./plot/PeriodMean/{tower[ti]}/Control_ContributionProfile_periodmean.png', bbox_inches='tight')
plt.close()

## Flux tower

In [2]:
tower = ['East', 'Flux', 'North', 'South_Mobile', 'West']
ti = 0
df3 = pd.ExcelFile(f'./data/PeriodMean/3m/2019_{tower[ti]}_Tower-fft_3mAGL.xlsx')
df10 = pd.ExcelFile(f'./data/PeriodMean/10m/2019_{tower[ti]}_Tower-fft_10mAGL.xlsx')
df20 = pd.ExcelFile(f'./data/PeriodMean/20m/2019_{tower[ti]}_Tower-fft_20mAGL.xlsx')

In [15]:
# # period mean 
# t = []
# w = []
# wt = []
# tmean = []
# wmean = []
# wtmean = []

# df = [df3, df10, df20]
# for hi in range(3):
#     df_pre = pd.read_excel(df[hi], 'Pre-FFP')
#     df_ffp = pd.read_excel(df[hi], 'FFP')
#     df_pos = pd.read_excel(df[hi], 'Post-FFP')

#     t.append(df_pre['T (C)'] - df_pre['T (C)'].mean())
#     t.append(df_ffp['T (C)'] - df_ffp['T (C)'].mean())
#     t.append(df_pos['T (C)'] - df_pos['T (C)'].mean())

#     w.append(df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())
#     w.append(df_ffp['W (m/s)'] - df_ffp['W (m/s)'].mean())
#     w.append(df_pos['W (m/s)'] - df_pos['W (m/s)'].mean())

#     wt.append((df_pre['T (C)'] - df_pre['T (C)'].mean()) * (df_pre['W (m/s)'] - df_pre['W (m/s)'].mean()))
#     wt.append((df_ffp['T (C)'] - df_ffp['T (C)'].mean()) * (df_ffp['W (m/s)'] - df_ffp['W (m/s)'].mean()))
#     wt.append((df_pos['T (C)'] - df_pos['T (C)'].mean()) * (df_pos['W (m/s)'] - df_pos['W (m/s)'].mean()))
    
#     tmean.append([df_pre['T (C)'].mean(), df_ffp['T (C)'].mean(), df_pos['T (C)'].mean()])
#     wmean.append([df_pre['W (m/s)'].mean(), df_ffp['W (m/s)'].mean(), df_pos['W (m/s)'].mean()])
#     wtmean.append([((df_pre['T (C)'] - df_pre['T (C)'].mean()) * (df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())).mean(),
#                   ((df_ffp['T (C)'] - df_ffp['T (C)'].mean()) * (df_ffp['W (m/s)'] - df_ffp['W (m/s)'].mean())).mean(),
#                   ((df_pos['T (C)'] - df_pos['T (C)'].mean()) * (df_pos['W (m/s)'] - df_pos['W (m/s)'].mean())).mean()])

# data = [t, w, wt]

In [3]:
# period mean for pre and pos and pre mean for ffp
t = []
w = []
wt = []
tmean = []
wmean = []
wtmean = []

df = [df3, df10, df20]
for hi in range(3):
    df_pre = pd.read_excel(df[hi], 'Pre-FFP')
    df_ffp = pd.read_excel(df[hi], 'FFP')
    df_pos = pd.read_excel(df[hi], 'Post-FFP')

    t.append(df_pre['T (C)'] - df_pre['T (C)'].mean())
    t.append(df_ffp['T (C)'] - df_pre['T (C)'].mean())
    t.append(df_pos['T (C)'] - df_pos['T (C)'].mean())

    w.append(df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())
    w.append(df_ffp['W (m/s)'] - df_pre['W (m/s)'].mean())
    w.append(df_pos['W (m/s)'] - df_pos['W (m/s)'].mean())

    wt.append((df_pre['T (C)'] - df_pre['T (C)'].mean()) * (df_pre['W (m/s)'] - df_pre['W (m/s)'].mean()))
    wt.append((df_ffp['T (C)'] - df_pre['T (C)'].mean()) * (df_ffp['W (m/s)'] - df_pre['W (m/s)'].mean()))
    wt.append((df_pos['T (C)'] - df_pos['T (C)'].mean()) * (df_pos['W (m/s)'] - df_pos['W (m/s)'].mean()))

    tmean.append([df_pre['T (C)'].mean(), df_ffp['T (C)'].mean(), df_pos['T (C)'].mean()])
    wmean.append([df_pre['W (m/s)'].mean(), df_ffp['W (m/s)'].mean(), df_pos['W (m/s)'].mean()])
    wtmean.append([((df_pre['T (C)'] - df_pre['T (C)'].mean()) * (df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())).mean(),
                  ((df_ffp['T (C)'] - df_ffp['T (C)'].mean()) * (df_ffp['W (m/s)'] - df_ffp['W (m/s)'].mean())).mean(),
                  ((df_pos['T (C)'] - df_pos['T (C)'].mean()) * (df_pos['W (m/s)'] - df_pos['W (m/s)'].mean())).mean()])
data = [t, w, wt]

In [35]:
# # mixed mean at west tower
# t = []
# w = []
# wt = []
# tmean = []
# wmean = []
# wtmean = []

# df = [df3, df10, df20]
# for hi in range(3):
#     df_pre = pd.read_excel(df[hi], 'Pre-FFP')[:18000]
#     df_ffp = pd.read_excel(df[hi], 'FFP')[:11988]
#     df_pos = pd.read_excel(df[hi], 'Post-FFP')[:18000]
#     if hi - 2:
#         t.append(df_pre['T (C)'] - df_pre['T (C)'].mean())
#         t.append(df_ffp['t(C)'] - df_pre['T (C)'].mean())
#         t.append(df_pos['t(C)'] - df_pos['t(C)'].mean())

#         w.append(df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())
#         w.append(df_ffp['w(m/s)'] - df_pre['W (m/s)'].mean())
#         w.append(df_pos['w(m/s)'] - df_pos['w(m/s)'].mean())

#         wt.append((df_pre['T (C)'] - df_pre['T (C)'].mean()) * (df_pre['W (m/s)'] - df_pre['W (m/s)'].mean()))
#         wt.append((df_ffp['t(C)'] - df_pre['T (C)'].mean()) * (df_ffp['w(m/s)'] - df_pre['W (m/s)'].mean()))
#         wt.append((df_pos['t(C)'] - df_pos['t(C)'].mean()) * (df_pos['w(m/s)'] - df_pos['w(m/s)'].mean()))
        
#         tmean.append([df_pre['T (C)'].mean(), df_ffp['t(C)'].mean(), df_pos['t(C)'].mean()])
#         wmean.append([df_pre['W (m/s)'].mean(), df_ffp['w(m/s)'].mean(), df_pos['w(m/s)'].mean()])
#         wtmean.append([((df_pre['T (C)'] - df_pre['T (C)'].mean()) * (df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())).mean(),
#                       ((df_ffp['t(C)'] - df_ffp['t(C)'].mean()) * (df_ffp['w(m/s)'] - df_ffp['w(m/s)'].mean())).mean(),
#                       ((df_pos['t(C)'] - df_pos['t(C)'].mean()) * (df_pos['w(m/s)'] - df_pos['w(m/s)'].mean())).mean()])
#     else:
#         t.append(df_pre['T (m/s)'] - df_pre['T (m/s)'].mean())
#         t.append(df_ffp['T (m/s)'] - df_pre['T (m/s)'].mean())
#         t.append(df_pos['T (m/s)'] - df_pos['T (m/s)'].mean())

#         w.append(df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())
#         w.append(df_ffp['W (m/s)'] - df_pre['W (m/s)'].mean())
#         w.append(df_pos['W (m/s)'] - df_pos['W (m/s)'].mean())

#         wt.append((df_pre['T (m/s)'] - df_pre['T (m/s)'].mean()) * (df_pre['W (m/s)'] - df_pre['W (m/s)'].mean()))
#         wt.append((df_ffp['T (m/s)'] - df_pre['T (m/s)'].mean()) * (df_ffp['W (m/s)'] - df_pre['W (m/s)'].mean()))
#         wt.append((df_pos['T (m/s)'] - df_pos['T (m/s)'].mean()) * (df_pos['W (m/s)'] - df_pos['W (m/s)'].mean()))
        
#         tmean.append([df_pre['T (m/s)'].mean(), df_ffp['T (m/s)'].mean(), df_pos['T (m/s)'].mean()])
#         wmean.append([df_pre['W (m/s)'].mean(), df_ffp['W (m/s)'].mean(), df_pos['W (m/s)'].mean()])
#         wtmean.append([((df_pre['T (m/s)'] - df_pre['T (m/s)'].mean()) * (df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())).mean(),
#                       ((df_ffp['T (m/s)'] - df_ffp['T (m/s)'].mean()) * (df_ffp['W (m/s)'] - df_ffp['W (m/s)'].mean())).mean(),
#                       ((df_pos['T (m/s)'] - df_pos['T (m/s)'].mean()) * (df_pos['W (m/s)'] - df_pos['W (m/s)'].mean())).mean()])

# data = [t, w, wt]

In [44]:
# # mixed mean at south tower
# t = []
# w = []
# wt = []
# tmean = []
# wmean = []
# wtmean = []

# df = [df3, [], df20]
# for hi in range(3):
#     if hi-1:
#         df_pre = pd.read_excel(df[hi], 'Pre-FFP')[:18000]
#         df_ffp = pd.read_excel(df[hi], 'FFP')[:11988]
#         df_pos = pd.read_excel(df[hi], 'Post-FFP')[:18000]
#     if hi - 1:
#         t.append(df_pre['T (C)'] - df_pre['T (C)'].mean())
#         t.append(df_ffp['t(C)'] - df_pre['T (C)'].mean())
#         t.append(df_pos['t(C)'] - df_pos['t(C)'].mean())

#         w.append(df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())
#         w.append(df_ffp['w(m/s)'] - df_pre['W (m/s)'].mean())
#         w.append(df_pos['w(m/s)'] - df_pos['w(m/s)'].mean())

#         wt.append((df_pre['T (C)'] - df_pre['T (C)'].mean()) * (df_pre['W (m/s)'] - df_pre['W (m/s)'].mean()))
#         wt.append((df_ffp['t(C)'] - df_pre['T (C)'].mean()) * (df_ffp['w(m/s)'] - df_pre['W (m/s)'].mean()))
#         wt.append((df_pos['t(C)'] - df_pos['t(C)'].mean()) * (df_pos['w(m/s)'] - df_pos['w(m/s)'].mean()))
        
#         tmean.append([df_pre['T (C)'].mean(), df_ffp['t(C)'].mean(), df_pos['t(C)'].mean()])
#         wmean.append([df_pre['W (m/s)'].mean(), df_ffp['w(m/s)'].mean(), df_pos['w(m/s)'].mean()])
#         wtmean.append([((df_pre['T (C)'] - df_pre['T (C)'].mean()) * (df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())).mean(),
#                       ((df_ffp['t(C)'] - df_ffp['t(C)'].mean()) * (df_ffp['w(m/s)'] - df_ffp['w(m/s)'].mean())).mean(),
#                       ((df_pos['t(C)'] - df_pos['t(C)'].mean()) * (df_pos['w(m/s)'] - df_pos['w(m/s)'].mean())).mean()])
#     else:
#         t.append([])
#         t.append([])
#         t.append([])
#         w.append([])
#         w.append([])
#         w.append([])
#         wt.append([])
#         wt.append([])
#         wt.append([])
#         tmean.append([np.nan, np.nan, np.nan])
#         wmean.append([np.nan, np.nan, np.nan])
#         wtmean.append([np.nan, np.nan, np.nan])
# data = [t, w, wt]

In [86]:
datamean = [tmean, wmean, wtmean]
color = ['r', 'b', 'lime']
start = ['15:08:00', '14:55:00', '15:43:00', '14:25:00', '14:55:00']
ffps = ['15:38:00', '15:25:00', '16:13:00', '14:55:00', '15:25:00']
ffpe = ['15:53:00', '15:40:00', '16:33:00', '15:15:00', '15:45:00']
end = ['16:23:00', '16:10:00', '17:03:00', '15:45:00', '16:15:00']

ylabel = ["t' (\u2103)", "w' (m/s)", "w't' (m\u2103/s)"]
yl = -5, -8, -100
yr = 95, 8, 300
step = 20, 4, 80

legend = ['19m AGL', '10m AGL', '3m AGL', 'Fire Starts', 'Fire Ends']
fig, axs = plt.subplots(3, 1, figsize=(15, 10))
for vi in range(3):
    ax = plt.subplot(3,1,vi+1)
    ax.set_ylim(yl[vi], yr[vi])
    plt.yticks(np.arange(yl[vi], yr[vi]+1, step[vi]), fontsize='large')
    ax.set_xlim(-10, 485)
    ax.set_ylabel(ylabel[vi], fontsize=15)
    plt.subplots_adjust(hspace=0.08)
    if vi == 2:
        plt.text(.5, -.16, 'Time', fontsize=15, fontweight='bold', c='k', ha='center', va='center', transform=ax.transAxes)
    for hi in [2,1,0]:
        wwpre = data[vi][3*hi]
        xpre = len(wwpre)

        wwffp = data[vi][3*hi+1]
        xffp = len(wwffp)

        wwpos = data[vi][3*hi+2]
        xpos = len(wwpos)

        plt.scatter(np.arange(xpre), wwpre, .5, color[hi], alpha=1.)
        plt.scatter(np.arange(xpre, xpre+xffp), wwffp, .5, color[hi], alpha=1., label='_nolegend_')
        plt.scatter(np.arange(xpre+xffp, xpre+xffp+xpos), wwpos, .5, color[hi], alpha=1., label='_nolegend_')
    
    if vi == 0:
        ht = .5
    else:
        ht = .1
    plt.text(.03, ht, meantext[vi], fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
    plt.text(.08, ht, f'{datamean[vi][0][0]:.3f}', fontsize=15, c=color[0], ha='center', va='center', transform=ax.transAxes)
    plt.text(.15, ht, f'{datamean[vi][1][0]:.3f}', fontsize=15, c=color[1], ha='center', va='center', transform=ax.transAxes)
    plt.text(.22, ht, f'{datamean[vi][2][0]:.3f}', fontsize=15, c=color[2], ha='center', va='center', transform=ax.transAxes)
    plt.text(.43, ht, f'{datamean[vi][0][1]:.3f}', fontsize=15, c=color[0], ha='center', va='center', transform=ax.transAxes)
    plt.text(.50, ht, f'{datamean[vi][1][1]:.3f}', fontsize=15, c=color[1], ha='center', va='center', transform=ax.transAxes)
    plt.text(.57, ht, f'{datamean[vi][2][1]:.3f}', fontsize=15, c=color[2], ha='center', va='center', transform=ax.transAxes)
    plt.text(.75, ht, f'{datamean[vi][0][2]:.3f}', fontsize=15, c=color[0], ha='center', va='center', transform=ax.transAxes)
    plt.text(.82, ht, f'{datamean[vi][1][2]:.3f}', fontsize=15, c=color[1], ha='center', va='center', transform=ax.transAxes)
    plt.text(.9, ht, f'{datamean[vi][2][2]:.3f}', fontsize=15, c=color[2], ha='center', va='center', transform=ax.transAxes)
    
    plt.vlines(xpre, yl[vi], yr[vi], 'magenta', linestyle='--', lw=1)
    plt.vlines(xpre+xffp, yl[vi], yr[vi], 'cyan', linestyle='--', lw=1) 
    plt.hlines(0, 0, xpre+xffp+xpos, 'grey', lw=1)
    plt.xticks(np.arange(0, xpre+xffp+xpos, 1000), [])
    if vi == 2:
        plt.text(0, -.08, start[ti], fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
#         plt.text(.21, -.08, mid1, fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
        plt.text(.41, -.08, ffps[ti], fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
        plt.text(.62, -.08, ffpe[ti], fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
#         plt.text(.82, -.08, mid2, fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
        plt.text(1, -.08, end[ti], fontsize=15, c='k', ha='center', va='center', transform=ax.transAxes)
    if vi == 0:
        plt.legend(legend, fontsize=14, markerscale=3., ncol=2, frameon=False)
    
plt.suptitle(f"{tower[ti]} Tower", fontsize=15, fontweight='bold', x=.5, y=.9)
plt.savefig(f'./plot/MixedMean/{tower[ti]}/{tower[ti]}_TimeSeries_mixedmean.png', bbox_inches='tight')
# plt.savefig(f'./plot/PeriodMean/{tower[ti]}/{tower[ti]}_TimeSeries_periodmean.png', bbox_inches='tight')
plt.close()

In [45]:
# w[3] = w[1] * np.nan
# w[4] = w[1] * np.nan
# w[5] = w[1] * np.nan
# t[3] = t[1] * np.nan
# t[4] = t[1] * np.nan
# t[5] = t[1] * np.nan

In [20]:
# for scatter plot
t = []
w = []

df = [df20, df10, df3]
for hi in range(3):
    df_pre = pd.read_excel(df[hi], 'Pre-FFP')
    df_ffp = pd.read_excel(df[hi], 'FFP')
    df_pos = pd.read_excel(df[hi], 'Post-FFP')

    t.append(df_pre['T (C)'] - df_pre['T (C)'].mean())
    t.append(df_ffp['T (C)'] - df_pre['T (C)'].mean())
    t.append(df_pos['T (C)'] - df_pos['T (C)'].mean())

    w.append(df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())
    w.append(df_ffp['W (m/s)'] - df_pre['W (m/s)'].mean())
    w.append(df_pos['W (m/s)'] - df_pos['W (m/s)'].mean())

data = [t, w, wt]

In [98]:
# # for scatter plot at west tower
# t = []
# w = []

# df = [df20, df10, df3]
# for hi in range(3):
#     df_pre = pd.read_excel(df[hi], 'Pre-FFP')[:18000]
#     df_ffp = pd.read_excel(df[hi], 'FFP')[:11988]
#     df_pos = pd.read_excel(df[hi], 'Post-FFP')[:18000]
#     if hi:
#         t.append(df_pre['T (C)'] - df_pre['T (C)'].mean())
#         t.append(df_ffp['t(C)'] - df_pre['T (C)'].mean())
#         t.append(df_pos['t(C)'] - df_pos['t(C)'].mean())

#         w.append(df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())
#         w.append(df_ffp['w(m/s)'] - df_pre['W (m/s)'].mean())
#         w.append(df_pos['w(m/s)'] - df_pos['w(m/s)'].mean())
#     else:
#         t.append(df_pre['T (m/s)'] - df_pre['T (m/s)'].mean())
#         t.append(df_ffp['T (m/s)'] - df_pre['T (m/s)'].mean())
#         t.append(df_pos['T (m/s)'] - df_pos['T (m/s)'].mean())

#         w.append(df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())
#         w.append(df_ffp['W (m/s)'] - df_pre['W (m/s)'].mean())
#         w.append(df_pos['W (m/s)'] - df_pos['W (m/s)'].mean())    

# data = [t, w]

In [98]:
# # for scatter plot at south tower
# t = []
# w = []

# # df = [df20, df10, df3]
# # df = [df20, [], df3]
# df = [df3, [], df20]
# for hi in range(3):
#     if hi - 1:
#         df_pre = pd.read_excel(df[hi], 'Pre-FFP')[:18000]
#         df_ffp = pd.read_excel(df[hi], 'FFP')[:11988]
#         df_pos = pd.read_excel(df[hi], 'Post-FFP')[:18000]
#     if hi - 1:
#         t.append(df_pre['T (C)'] - df_pre['T (C)'].mean())
#         t.append(df_ffp['t(C)'] - df_pre['T (C)'].mean())
#         t.append(df_pos['t(C)'] - df_pos['t(C)'].mean())

#         w.append(df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())
#         w.append(df_ffp['w(m/s)'] - df_pre['W (m/s)'].mean())
#         w.append(df_pos['w(m/s)'] - df_pos['w(m/s)'].mean())
#     else:
# #         t.append(df_pre['T (m/s)'] - df_pre['T (m/s)'].mean())
# #         t.append(df_ffp['T (m/s)'] - df_pre['T (m/s)'].mean())
# #         t.append(df_pos['T (m/s)'] - df_pos['T (m/s)'].mean())

# #         w.append(df_pre['W (m/s)'] - df_pre['W (m/s)'].mean())
# #         w.append(df_ffp['W (m/s)'] - df_pre['W (m/s)'].mean())
# #         w.append(df_pos['W (m/s)'] - df_pos['W (m/s)'].mean())   
#         t.append(t[0]*np.nan)
#         t.append(t[1]*np.nan)
#         t.append(t[2]*np.nan)
#         w.append(w[0]*np.nan)
#         w.append(w[1]*np.nan)
#         w.append(w[2]*np.nan)

# data = [t, w]

In [4]:
SweepC = [np.logical_and(w[i]<0, t[i]<0).sum() for i in range(9)]
EjectionC = [np.logical_and(w[i]>0, t[i]>0).sum() for i in range(9)]
OutwardC = [np.logical_and(w[i]>0, t[i]<0).sum() for i in range(9)]
InwardC = [np.logical_and(w[i]<0, t[i]>0).sum() for i in range(9)]

TotalC = [SweepC[i]+EjectionC[i]+OutwardC[i]+InwardC[i] for i in range(9)]
SweepMean = [np.where(np.logical_and(w[i]<0, t[i]<0), w[i]*t[i], 0).sum() / TotalC[i] for i in range(9)]
EjectionMean = [np.where(np.logical_and(w[i]>0, t[i]>0), w[i]*t[i], 0).sum() / TotalC[i] for i in range(9)]
OutwardMean = [np.where(np.logical_and(w[i]>0, t[i]<0), w[i]*t[i], 0).sum() / TotalC[i] for i in range(9)]
InwardMean = [np.where(np.logical_and(w[i]<0, t[i]>0), w[i]*t[i], 0).sum() / TotalC[i] for i in range(9)]

OutwardFra = [OutwardC[i] / TotalC[i] for i in range(9)]
EjectionFra = [EjectionC[i] / TotalC[i] for i in range(9)]
InwardFra = [InwardC[i] / TotalC[i] for i in range(9)]
SweepFra = [SweepC[i] / TotalC[i] for i in range(9)]

In [5]:
[OutwardFra[i] + InwardFra[i] + EjectionFra[i] + SweepFra[i] for i in range(9)]

[1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999999999999, 1.0, 1.0, 1.0]

In [6]:
SF = np.array(SweepFra)
EF = np.array(EjectionFra)
OF = np.array(OutwardFra)
IF = np.array(InwardFra)
HeatMean = np.array(SweepMean) + np.array(EjectionMean) + np.array(InwardMean) + np.array(OutwardMean)

contriF = [SF, EF, OF, IF, HeatMean]
PerChge = np.zeros((6, 5))
for ei in range(5):
    for hi in range(3):
        PerChge[hi*2, ei] = (contriF[ei][hi*3+1] - contriF[ei][hi*3]) / contriF[ei][hi*3] * 100
        PerChge[hi*2+1, ei] = (contriF[ei][hi*3+2] - contriF[ei][hi*3]) / contriF[ei][hi*3] * 100

In [7]:
pd.DataFrame({'Sweep':SF, 'Ejection':EF, 'Outward':OF, 'Inward':IF, 'Total':HeatMean}).T.round(2)

,0,1,2,3,4,5,6,7,8
Sweep,0.33,0.21,0.30,0.33,0.28,0.23,0.34,0.40,0.25
Ejection,0.29,0.43,0.26,0.29,0.47,0.27,0.30,0.40,0.31
Outward,0.22,0.08,0.23,0.23,0.09,0.27,0.22,0.11,0.23
Inward,0.16,0.28,0.21,0.14,0.16,0.22,0.14,0.08,0.21
Total,0.09,2.20,0.06,0.20,3.13,0.06,0.40,4.43,0.18


In [10]:
def color_PositiveNegative(s):
    if s[0] == '-':
        color = 'blue'
    else:
        color = 'red'
    return 'color: %s' % color

In [51]:
table = pd.DataFrame({'Sweep':PerChge[:, 0], 'Ejection':PerChge[:, 1], 'Outward':PerChge[:, 2], 'Inward':PerChge[:, 3], 'Total':PerChge[:, 4]}).T.round(2).astype(str)
table.style.applymap(color_PositiveNegative)

,0,1,2,3,4,5
Sweep,-80.8,-2.66,nan,nan,-70.34,1.85
Ejection,82.64,7.19,nan,nan,86.17,4.36
Outward,-84.31,-5.22,nan,nan,-73.78,-27.48
Inward,174.39,-3.01,nan,nan,142.52,25.94
Total,2287.54,21.32,nan,nan,1149.02,36.91


In [101]:
color = ['lime', 'r', 'k']
xl = [-6, -20, -6]
xr = [12, 100, 12]
yl = [-8, -8, -8]
yr = [8, 8, 8]
xstep = [3, 20, 3]
ystep = [4, 4, 4]
fig, axs = plt.subplots(3, 3, figsize=(12, 12))
for hi in range(3):
    for pi in range(3):
        ax = plt.subplot(3,3,hi*3+pi+1)
        plt.scatter(t[hi*3+pi], w[hi*3+pi], c=color[pi], s=1)
        plt.xlim(xl[pi], xr[pi])
        plt.ylim(yl[pi], yr[pi])
        plt.hlines(0, xl[pi], xr[pi], 'c', lw=1)
        plt.vlines(0, yl[pi], yr[pi], 'c', lw=1)
        plt.subplots_adjust(hspace=.06, wspace=.085)
        if pi == 0:
            plt.ylabel("$w' (ms^{-1})$", fontsize=15)
            plt.yticks(np.arange(yl[pi], yr[pi]+ystep[pi], ystep[pi]), fontsize=13)
        else:
            plt.yticks(np.arange(yl[pi], yr[pi]+ystep[pi], ystep[pi]), [])
        if hi == 2:
            plt.xlabel("t' (\u2103)", fontsize=15)
            plt.xticks(np.arange(xl[pi], xr[pi]+xstep[pi], xstep[pi]), fontsize=13)
        else:
            plt.xticks(np.arange(xl[pi], xr[pi]+xstep[pi], xstep[pi]), [])
        if hi == 0:
            if pi == 0:
                plt.text(.5, 1.08, 'Pre-FFP', fontsize='large', fontweight='bold', c='lime', ha='center', va='center', transform=ax.transAxes)
            elif pi == 1:
                plt.text(.5, 1.08, 'FFP', fontsize='large', fontweight='bold', c='r', ha='center', va='center', transform=ax.transAxes)
            elif pi == 2:
                plt.text(.5, 1.08, 'Post-FFP', fontsize='large', fontweight='bold', c='k', ha='center', va='center', transform=ax.transAxes)
        
        if pi == 0:
            if hi == 0:
                plt.text(-.17, .9, '19-m', fontsize=15, ha='center', va='center', transform=ax.transAxes)
            elif hi == 1:
                plt.text(-.17, .9, '10-m', fontsize=15, ha='center', va='center', transform=ax.transAxes)
            elif hi == 2:
                plt.text(-.17, .9, '3-m', fontsize=15, ha='center', va='center', transform=ax.transAxes)
        
        plt.text(.25, .96, 'Outward', c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.23, .89, "$\overline{w^{'}t^{'}}$ = "+f"{OutwardMean[hi*3+pi]:.4f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.23, .81, f" Fraction= {OutwardFra[hi*3+pi]:.2f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.25, .2, 'Sweep', c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.23, .12, "$\overline{w^{'}t^{'}}$ = "+f"{SweepMean[hi*3+pi]:.4f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.23, .05, f" Fraction= {SweepFra[hi*3+pi]:.2f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.77, .2, 'Inward', c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.77, .12, "$\overline{w^{'}t^{'}}$ = "+f"{InwardMean[hi*3+pi]:.4f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.75, .05, f" Fraction= {InwardFra[hi*3+pi]:.2f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.77, .96, 'Ejection', c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.77, .89, "$\overline{w^{'}t^{'}}$ = "+f"{EjectionMean[hi*3+pi]:.4f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
        plt.text(.75, .81, f" Fraction= {EjectionFra[hi*3+pi]:.2f}", c='b', fontsize=14, ha='center', va='center', transform=ax.transAxes)
plt.suptitle(f'{tower[ti]} tower', fontsize='large', fontweight='bold', x=.5, y=.92)
plt.savefig(f'./plot/MixedMean/{tower[ti]}/{tower[ti]}_scatterplot_mixedmean.png', bbox_inches='tight')
# plt.savefig(f'./plot/PeriodMean/{tower[ti]}/{tower[ti]}_scatterplot_periodmean.png', bbox_inches='tight')
plt.close()

In [97]:
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
Y = [3, 10, 19]
HeatMean = np.array(SweepMean) + np.array(EjectionMean) + np.array(InwardMean) + np.array(OutwardMean)
for pi in range(3):
    ax = plt.subplot(1,3,pi+1)
    plt.subplots_adjust(wspace=.06)
    plt.barh(Y, np.array(SweepMean[pi::3])+np.array(EjectionMean[pi::3]), align='center', height=2, color='pink')
    plt.barh(Y, np.array(SweepMean[pi::3]), align='center', height=2, color='red')
    plt.barh(Y, np.array(InwardMean[pi::3])+np.array(OutwardMean[pi::3]), align='center', height=2, color='b')
    plt.barh(Y, np.array(InwardMean[pi::3]), align='center', height=2, color='cyan')
    plt.vlines(HeatMean[pi], 1.5, 4.5, lw=1, colors='yellow')
    plt.text(HeatMean[pi]-.018, 4.7, f'{HeatMean[pi]:.3f}', fontsize=14)
    plt.vlines(HeatMean[pi+3], 8.5, 11.5, lw=1, colors='yellow')
    plt.text(HeatMean[pi+3]-.018, 11.7, f'{HeatMean[pi+3]:.3f}', fontsize=14)
    plt.vlines(HeatMean[pi+6], 17.5, 20.5, lw=1, colors='yellow')
    plt.text(HeatMean[pi+6]-.018, 20.7, f'{HeatMean[pi+6]:.3f}', fontsize=14)
    plt.vlines(0, 0, 22, lw=1, ls='dashed', color='grey')
    if pi-1:
        plt.xlim(-.3, .7)
        plt.xticks([i*.1 for i in range(-3, 7)], ['', '-0.2', '', '0.0', '', '0.2', '', '0.4', '', '0.6', ''], fontsize=14)
    else:
        plt.xlim(-1, 5)
        plt.xticks(np.arange(-1, 6, 1), fontsize=14)
    plt.xlabel("$w't' (\u2103ms^{-1})$", fontsize=15)
    plt.ylim(0, 22)
    if pi == 0:
        plt.ylabel("Height (m)", fontsize=15)
        plt.yticks([i for i in range(0, 22, 3)], fontsize=14)
    else:
        plt.yticks([i for i in range(0, 22, 3)], [])
    
    if pi == 0:
        plt.text(.5, 1.2, 'Pre-FFP', fontsize=15, fontweight='bold', c='lime', ha='center', va='center', transform=ax.transAxes)
    elif pi == 1:
        plt.text(.5, 1.2, 'FFP', fontsize=15, fontweight='bold', c='r', ha='center', va='center', transform=ax.transAxes)
    elif pi == 2:
        plt.text(.5, 1.2, 'Post-FFP', fontsize=15, fontweight='bold', c='k', ha='center', va='center', transform=ax.transAxes)
    
    axx = ax.twiny()
    axx.plot(SweepFra[pi::3], Y, 'r-o', ms=3)
    axx.scatter(SweepFra[pi::3], Y, color='m')
    axx.plot(EjectionFra[pi::3], Y, c='pink', ls='-', marker='o', ms=3)
    axx.scatter(EjectionFra[pi::3], Y, color='m')
    axx.plot(InwardFra[pi::3], Y, c='cyan', ls='-', marker='o', ms=3)
    axx.scatter(InwardFra[pi::3], Y, color='m')
    axx.plot(OutwardFra[pi::3], Y, 'b-o', ms=3)
    axx.scatter(OutwardFra[pi::3], Y, color='m')
    axx.set_xlim(0, .7)
    axx.set_xticks(np.arange(0, 0.7, 0.1))
    axx.set_xticklabels(['0.0', '', '0.2', '', '0.4', '', '0.6', ''], fontsize=14)
    axx.set_xlabel('Fraction of Events', fontsize=15)
    axx.set_yticks(np.arange(0,22,3))
    plt.legend(['Sweep', 'Ejection', 'Inward', 'Outward'], loc='lower right', frameon=False, fontsize=14)
plt.suptitle(f'{tower[ti]} tower', fontsize=15, fontweight='bold', x=.5, y=1.12)
plt.savefig(f'./plot/MixedMean/{tower[ti]}/{tower[ti]}_ContributionProfile_mixedmean.png', bbox_inches='tight')
# plt.savefig(f'./plot/PeriodMean/{tower[ti]}/{tower[ti]}_ContributionProfile_periodmean.png', bbox_inches='tight')
plt.close()

posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
